In [7]:
import os
import numpy as np
import pandas as pd
import re

df_target = pd.read_csv("y-enjoyment.csv")

X = []
y = []
person_ids = []
for file in os.listdir("data/text-datasets/sfr-mistral/text-embeddings-short"):
    if file.endswith("csv"):
        # search for number in filename
        match = re.search(r"(\d+)", file)
        if match is None:
            continue
        person_id = int(match.group(1))
        person_ids.append(person_id)
        df = pd.read_csv(
            os.path.join("data/text-datasets/sfr-mistral/text-embeddings-short", file), header=0
        )
        X_aux = df.iloc[:, 1:].values
        X_aux = np.mean(X_aux, axis=0)
        X.append(X_aux)
        y.append(df_target[df_target["user_id"] == person_id]["Average"])

# sort by person_id
sorted_indices = np.argsort(person_ids)
X = [X[i] for i in sorted_indices]
y = [y[i] for i in sorted_indices]
person_ids = [person_ids[i] for i in sorted_indices]
# convert to numpy arrays
X = np.vstack(X)
y = np.vstack(y)
print(X.shape)
print(y.shape)

(38, 4096)
(38, 1)


In [ ]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import Ridge, LinearRegression # Or RidgeCV for automatic alpha tuning
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingRegressor
import torch

def CCCLoss(x, y):
    """
    Calculates the Concordance Correlation Coefficient (CCC) Loss.
    CCC measures the agreement between two variables.
    Loss = 1 - CCC, because loss functions are minimized, while CCC is maximized.
    """
    # Check if inputs are tensors
    if not isinstance(x, torch.Tensor) or not isinstance(y, torch.Tensor):
        # Convert inputs to tensors
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
    # Ensure inputs are flat tensors
    x = x.view(-1)
    y = y.view(-1)

    # Calculate means
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)

    # Calculate variances using population variance (unbiased=False)
    var_x = torch.var(x, unbiased=False)
    var_y = torch.var(y, unbiased=False)

    # Calculate covariance between x and y (population covariance)
    cov_xy = torch.mean((x - mean_x) * (y - mean_y))

    # Calculate CCC
    numerator = 2 * cov_xy
    denominator = var_x + var_y + (mean_x - mean_y)**2

    # Add a small epsilon for numerical stability (prevents division by zero)
    epsilon = 1e-8
    ccc = numerator / (denominator + epsilon)

    # Return 1 - CCC because loss functions should be minimized
    return 1.0 - ccc

# Initialize LeaveOneOut
loo = LeaveOneOut()

# Lists to store actual values and predictions from each fold
y_true_all = []
y_pred_all = []

print(f"Starting LOOCV for {len(y)} samples...")

# Assuming X and y are your input data and target values
X, y, person_ids = shuffle(X, y, person_ids) # Shuffle data with a fixed random seed

# Loop through LOOCV splits
for train_index, test_index in loo.split(X):

    model_to_use = LinearRegression() # cv=5 for inner loop tuning

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the Ridge model
    model_to_use.fit(X_train, y_train)

    # Predict on the left-out sample
    y_pred = model_to_use.predict(X_test)

    # Store the actual and predicted values
    y_true_all.append(y_test[0])
    y_pred_all.append(y_pred[0])

print("LOOCV finished.")

# Convert lists to numpy arrays for metric calculation
y_true_all = np.array(y_true_all).flatten()
y_pred_all = np.array(y_pred_all).flatten()

# Calculate final metrics based on aggregated predictions
mse = mean_squared_error(y_true_all, y_pred_all)
r2 = r2_score(y_true_all, y_pred_all)
correlation, p_value = pearsonr(y_true_all, y_pred_all)

print(f"\n--- LOOCV Results (N={len(y_true_all)}) ---")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R2): {r2:.4f}")
print(f"Pearson Correlation: {correlation:.4f}")
print(f"P-value for Correlation: {p_value:.4f}")

# Save the predictions to a CSV file
predictions_df = pd.DataFrame({
    'Person ID': person_ids,
    'True Values': y_true_all,
    'Predicted Values': y_pred_all.round(2),
})
# Order by Person ID
predictions_df.sort_values(by='Person ID', inplace=True)
modality_suffix = "text"
predictions_df.to_csv(f'predictions-sfr-mistral-mean.csv', index=False)

#compute MAE
mae = np.mean(np.abs(y_true_all - y_pred_all))
print(f"Mean Absolute Error (MAE): {mae:.4f}")
#compute CCC
ccc = 1 - CCCLoss(torch.tensor(y_true_all), torch.tensor(y_pred_all))
print(f"Concordance Correlation Coefficient (CCC): {ccc:.4f}")

Starting LOOCV for 38 samples...
LOOCV finished.

--- LOOCV Results (N=38) ---
Mean Squared Error (MSE): 1.3241
R-squared (R2): 0.2913
Pearson Correlation: 0.5769
P-value for Correlation: 0.0001
Mean Absolute Error (MAE): 0.9564
Concordance Correlation Coefficient (CCC): 0.5560
